In [1]:
import os
import sys
import pandas as pd
import yfinance as yf
import plotly as px

In [2]:
import os, sys
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from statsmodels.tsa.seasonal import seasonal_decompose
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import word_tokenize

In [3]:


# Get theconda create --name talib python=3.9 current working directory
current_dir = os.getcwd()
print(current_dir)

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Insert the path to the parent directory
sys.path.insert(0, parent_dir)

# # Insert the path to the Scripts directory
# sys.path.insert(0, os.path.join(parent_dir, 'Scripts'))

# print(sys.path)

d:\Kifya\Week01\notebooks
d:\Kifya\Week01


In [4]:
# nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\asmam\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [5]:
news_data = pd.read_csv('../yfinance_data//raw_analyst_ratings.csv')
# Remove timezone information and convert to datetime

# Step 1: Convert the 'date' column to datetime format
news_data['date'] = pd.to_datetime(news_data['date'], errors='coerce')

# Step 2: Remove timezone information (if any)
news_data['local_date'] = news_data['date'].dt.tz_localize(None)
news_data['date_only'] = news_data['date'].dt.date
news_data

,Unnamed: 0,headline,url,publisher,date,stock,local_date,date_only
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A,2020-06-05 10:30:54,2020-06-05
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A,2020-06-03 10:45:20,2020-06-03
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A,2020-05-26 04:30:07,2020-05-26
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A,2020-05-22 12:45:06,2020-05-22
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A,2020-05-22 11:38:59,2020-05-22
...,...,...,...,...,...,...,...,...
1407323,1413844,Top Narrow Based Indexes For August 29,https://www.benzinga.com/news/11/08/1888782/to...,Monica Gerson,NaT,ZX,NaT,NaT
1407324,1413845,Recap: Wednesday's Top Percentage Gainers and ...,https://www.benzinga.com/news/earnings/11/06/1...,Benjamin Lee,NaT,ZX,NaT,NaT
1407325,1413846,UPDATE: Oppenheimer Color on China Zenix Auto ...,https://www.benzinga.com/analyst-ratings/analy...,BenzingaStaffL,NaT,ZX,NaT,NaT
1407326,1413847,Oppenheimer Initiates China Zenix At Outperfor...,https://www.benzinga.com/analyst-ratings/price...,Joe Young,NaT,ZX,NaT,NaT


In [6]:
# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()
# Calculate the sentiment of the headlines
news_data['sentiment'] = news_data['headline'].apply(lambda x: sia.polarity_scores(text=x)['compound'])


# Initialize VADER sentiment analyzer
# Categorize the sentiment scores
news_data['sentiment_category'] = pd.cut(news_data['sentiment'], bins=[-1, -0.5, -0.0001, 0.5, 1], labels=['Very Negative', 'Negative', 'Neutral', 'Positive'])
news_data.head()

,Unnamed: 0,headline,url,publisher,date,stock,local_date,date_only,sentiment,sentiment_category
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A,2020-06-05 10:30:54,2020-06-05,0.000,Neutral
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A,2020-06-03 10:45:20,2020-06-03,0.000,Neutral
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A,2020-05-26 04:30:07,2020-05-26,0.000,Neutral
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A,2020-05-22 12:45:06,2020-05-22,0.000,Neutral
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A,2020-05-22 11:38:59,2020-05-22,0.296,Neutral


In [8]:
import pandas as pd

def count_stocks(news_data, stocks):

    stock_counts = {}
    for stock in stocks:
        stock_counts[stock] = len(news_data[news_data['stock'] == stock])
    return stock_counts

# Example usage:
stocks_to_count = ['AAPL', 'GOOG', 'MSFT', 'META','AMZN','NVDA','TSLA']
stock_counts = count_stocks(news_data, stocks_to_count)
print(stock_counts)

{'AAPL': 441, 'GOOG': 1199, 'MSFT': 0, 'META': 0, 'AMZN': 278, 'NVDA': 3146, 'TSLA': 1875}


exclude 'MSFT': 0, 'META': 0,for correlation

In [9]:
news_data = news_data.set_index('date_only')
news_data.head(2)

,Unnamed: 0,headline,url,publisher,date,stock,local_date,sentiment,sentiment_category
date_only,,,,,,,,,
2020-06-05,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A,2020-06-05 10:30:54,0.0,Neutral
2020-06-03,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A,2020-06-03 10:45:20,0.0,Neutral


In [15]:
news_data=news_data.drop(columns='Unnamed: 0', axis=1)
news_data.head(1)

,headline,url,publisher,date,stock,local_date,sentiment,sentiment_category
date_only,,,,,,,,
2020-06-05,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A,2020-06-05 10:30:54,0.0,Neutral


In [29]:
news_data.reset_index(inplace=True)

In [30]:
news_data['date'] = pd.to_datetime(news_data['date'], errors='coerce')

# Step 2: Remove timezone information (if any)
# news_data['local_date'] = news_data['date'].dt.tz_localize(None)
news_data['Date'] = news_data['date'].dt.date

In [31]:
news_data.head(1)

,index,date_only,headline,url,publisher,date,stock,local_date,sentiment,sentiment_category,Date
0,0,2020-06-05,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A,2020-06-05 10:30:54,0.0,Neutral,2020-06-05


In [32]:
# news_data['Date'] = pd.to_datetime(news_data['date_only'])
news_data['Date']=news_data.set_index('Date', inplace=True)
news_data.head(1)

,index,date_only,headline,url,publisher,date,stock,local_date,sentiment,sentiment_category,Date
Date,,,,,,,,,,,
2020-06-05,0,2020-06-05,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A,2020-06-05 10:30:54,0.0,Neutral,None


### Stocks data

In [18]:

# Load data
tesla_df = pd.read_csv('../yfinance_data/TSLA_historical_data.csv')
goog_df = pd.read_csv('../yfinance_data/GOOG_historical_data.csv')
amzn_df = pd.read_csv('../yfinance_data/AMZN_historical_data.csv')
nvda_df = pd.read_csv('../yfinance_data/NVDA_historical_data.csv')
aapl_df = pd.read_csv('../yfinance_data/AAPL_historical_data.csv')

In [ ]:
# Convert Date columns to datetime
amzn_df['Date'] = pd.to_datetime(amzn_df['Date'])
tesla_df['Date'] = pd.to_datetime(tesla_df['Date'])
goog_df['Date'] = pd.to_datetime(goog_df['Date'])
nvda_df['Date'] = pd.to_datetime(nvda_df['Date'])
aapl_df['Date'] = pd.to_datetime(aapl_df['Date'])

In [ ]:





# Set Date as index
meta_df.set_index('Date', inplace=True)
tesla_df.set_index('Date', inplace=True)
goog_df.set_index('Date', inplace=True)

# Rename columns to differentiate between META, TSLA, and GOOG
meta_df.columns = [f'META_{col}' for col in meta_df.columns]
tesla_df.columns = [f'TSLA_{col}' for col in tesla_df.columns]
goog_df.columns = [f'GOOG_{col}' for col in goog_df.columns]

# Concatenate all DataFrames
merged_data = pd.concat([meta_df, tesla_df, goog_df], axis=1, join='inner')